This is the global parameters that are set. This includes the subject and the file directories. The PAR.subjects value needs to be changed to match which subject you want to analyze

In [ ]:
% Batch mode scripts for running spm5 in TRC
% Created by Ze Wang, 08-05-2004
% zewang@mail.med.upenn.edu
% Adjusted by Marc Lindley for Lab class

addpath(genpath(pwd));

fprintf('\r%s\n',repmat(sprintf('-'),1,30))
fprintf('%-40s\n','Set PAR')


clear
global PAR
PAR=[];


PAR.SPM_path=spm('Dir');
addpath(PAR.SPM_path);

% This file sets up various things specific to this
% analysis, and stores them in the global variable PAR,
% which is used by the other batch files.
% You don't have to do it this way of course, I just
% found it easier



%%%%%%%%%%%%%%%%%%%%%
%                   %
%   GENERAL PREFS   %
%                   %
%%%%%%%%%%%%%%%%%%%%%%
% Where the subjects' data directories are stored

PAR.batchcode_which= mfilename('fullpath');
PAR.batchcode_which=fileparts(PAR.batchcode_which);
addpath(PAR.batchcode_which);
old_pwd=pwd;
cd(pwd);
cd ../
data_root=pwd;
cd(old_pwd);


PAR.root=data_root;

% This needs to be adjusted to include the subject names
% Subjects' directories
PAR.subjects = {'sub1' } ;
PAR.nsubs = length(PAR.subjects);


% Ensure this reflects the actual directory name
% Anatomical directory name
PAR.structfilter='Anatomy';


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Get the anatomical image directories automatically
for sb=1:PAR.nsubs
    tmp=dir(fullfile(PAR.root,PAR.subjects{sb},['*' PAR.structfilter '*']));
    if size(tmp,1)==0
        sprintf('Can not find the anatomical directory for subject\n')
        sprintf('%s: \n', char(PAR.subjects{sb}))
        error('Can not find the anatomical directory for subject');
    end
    if size(tmp,1)>1
        sprintf('More than 1 anatomical directories for subject: %s are found here!\n',char(PAR.subjects{sb}))
        error('More than 1 anatomical directories are found')
    end
    PAR.structdir{sb}=fullfile(PAR.root,PAR.subjects{sb},spm_str_manip(char(tmp(1).name),'d'));
end
%prefixes for filenames of structural 3D images, supposed to be the same for every subj.
PAR.structprefs = 'Anatomy';
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% Getting condition directories
PAR.confilters={'ASL_S1'}; %filter for folder name and filenames of functional images
PAR.ncond=length(PAR.confilters);
PAR.M0filters = {'ASL_S1M0'};

% The condition names are assumed same for different sessions

for sb=1:PAR.nsubs
    for c=1:PAR.ncond
        tmp=dir(fullfile(PAR.root,PAR.subjects{sb},['*' PAR.confilters{c} ]));

        if size(tmp,1)==0
            sprintf('Can not find the condition directory for subject\n')
            sprintf('%s: \n', char(PAR.subjects{sb}))
            error('Can not find the condition directory for subject');
        end

        if size(tmp,1)>1
            sprintf('Panic! subject %s has more than 1 directories!\n', [PAR.subjects{sb}])
            error('Panic! condition has more than 1 directories!')
            %return;
        end
        PAR.condirs{sb,c}=fullfile(PAR.root,PAR.subjects{sb},spm_str_manip(char(tmp(1).name),'d'));
        
        
          tmp=dir(fullfile(PAR.root,PAR.subjects{sb},['*' PAR.M0filters{c} ]));
             if size(tmp,1)==0
            sprintf('Can not find the M0 directory for subject\n')
            sprintf('%s: \n', char(PAR.subjects{sb}))
            error('Can not find the M0 directory for subject');
        end

        if size(tmp,1)>1
            sprintf('Panic! subject %s has more than 1 M0 directories!\n', [PAR.subjects{sb}])
            error('Panic! condition has more than 1 M0 directories!')
            %return;
        end
        
          PAR.M0dirs{sb,c}=fullfile(PAR.root,PAR.subjects{sb},spm_str_manip(char(tmp(1).name),'d'));
          
    end
end

% Smoothing kernel size
PAR.FWHM = [6];

% % TR for each subject.  As one experiment was carried out in one Hospital (with one machine)
% % and the other in another hospital (different machine), TRs are slightly different
% %PAR.TRs = [2.4696 2];
PAR.TRs = ones(1,PAR.nsubs)*6;
% PAR.mp='no';

% % Model stuff
% % Condition namesfiles
PAR.cond_names = {'rest','tap'};
PAR.condnum=length(PAR.cond_names);
PAR.Onsets={[0 24 48]
    [12 36 60]};

PAR.Durations={[12]
    [12]};

%
PAR.mp='no';
%
PAR.groupdir = ['group_anasmallPerf_sinc'];

%contrast names
PAR.con_names={'tap_rest'};


PAR.subtractiontype=0;
PAR.glcbffile=['globalsg_' num2str(PAR.subtractiontype) '.txt'];
PAR.img4analysis='cbf'; % or 'Perf'
PAR.ana_dir = ['glm_' PAR.img4analysis];
PAR.Filter='cbf_0_sr';


Reset orientation

In [ ]:
% batch reset T1 orientation
global PAR
fprintf('\r%-40s\n','batch reset orientation ')
%%%%% the following codes are changed from batch_smooth

% dirnames,
% get the subdirectories in the main directory
for sb = 1:PAR.nsubs % for each subject
    %go get the sessions
    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb} );
    fprintf('\r%-40s  %30s',str,' ')
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...reseting')  %-#
    %now get all images to reset orientation
    %prepare directory
    P=[];
    for c=1:PAR.ncond
        % get files in this directory
        Ptmp=spm_select('FPList', char(PAR.condirs{sb,c}), ['^' PAR.confilters{c} '\w*\.img$']);

        P=strvcat(P,Ptmp);
        
        
        if strcmp(P(1,:),'/')
            disp('!!!!!!!!\n NO epi file\n!!!!!!!!!!')
            continue;
        end;

        
        Ptmp=spm_select('FPList', char(PAR.M0dirs{sb,c}), ['^' PAR.M0filters{c} '\w*\.img$']);
        
        
        P=strvcat(P,Ptmp);
        
        
        for i=1:size(P,1)
            fprintf('%s%30s',repmat(sprintf('\b'),1,30),sprintf('%s%5d/%5d','...reseting epi ',i,size(P,1)) ) %-#

            V    = spm_vol(deblank(P(i,:)));
            M    = V.mat;
            vox  = sqrt(sum(M(1:3,1:3).^2));
            if det(M(1:3,1:3))<0, vox(1) = -vox(1); end;
            orig = (V.dim(1:3)+1)/2;
            off  = -vox.*orig;
            M    = [vox(1)  0      0      off(1)
                0      vox(2) 0      off(2)
                0      0      vox(3) off(3)
                0      0      0      1       ];
            spm_get_space(P(i,:),M);
        end
    end

    P=[];
    Ptmp=spm_select('FPList', char(PAR.structdir{sb}), ['^' PAR.structprefs '\w*.img$']);

    P=strvcat(P,Ptmp);
    if strcmp(P(1,:),'/')
        disp('!!!!!\n NO struct file\n!!!!!!!!!!!!!!!')
        continue;
    end;

    for i=1:size(P,1),
        fprintf('%s%30s',repmat(sprintf('\b'),1,30),sprintf('%s%5d/%5d','...reseting T1 ',i,size(P,1)) ) %-#
        V    = spm_vol(deblank(P(i,:)));
        M    = V.mat;
        vox  = sqrt(sum(M(1:3,1:3).^2));
        if det(M(1:3,1:3))<0, vox(1) = -vox(1); end;
        orig = (V.dim(1:3)+1)/2;
        off  = -vox.*orig;
        M    = [vox(1)  0      0      off(1)
                 0      vox(2) 0      off(2)
                 0      0      vox(3) off(3)
                 0      0      0      1   ];
        spm_get_space(P(i,:),M);
    end;

    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...done')  %-#
end


In [ ]:
img=load_nii(Ptmp)

In [ ]:
imatlab_export_fig('print-png')
imagesc(img.img(:,:,40))

Set the Origin

In [ ]:
fprintf('%s\n',repmat(sprintf('-'),1,30))
fprintf('Set Origin\n');

% epiorg   functional origins
% t1org      t1 origins

PAR.subs(1).cond(1).epiorg= [-3.1 21.8 -4.4];      %sub1
PAR.subs(1).t1org = [-2.9 22.5 -2.9];

PAR.subs(2).cond(1).epiorg= [-3.4 13.0 -13.7];      %sub2
PAR.subs(2).t1org = [-2.9 4.0 -12.3];

PAR.subs(3).cond(1).epiorg= [-1.7 22.5 -10.2];      %sub3
PAR.subs(3).t1org = [0.0 30.1 -14.9];



for sb = 1:PAR.nsubs % for each subject
    %go get the sessions
    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb});
    fprintf('\r%-40s  %30s',str,' ')
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...seting epi')  %-#

    %now get all images to reset orientation
    %prepare directory
    P=[];
    for c=1:PAR.ncond
        
        % get files in this directory
        Ptmp=spm_select('FPList', char(PAR.condirs{sb,c}), ['^'  PAR.confilters{c} '\w*\.img$']);
        P=strvcat(P,Ptmp);
          Ptmp=spm_select('FPList', char(PAR.M0dirs{sb,c}), ['^' PAR.M0filters{c} '\w*\.img$']);
        P=strvcat(P,Ptmp);
        
        
        if strcmp(P(1,:),'/'),continue;end;
        
        if ~isempty(PAR.subs(sb).cond(c).epiorg)
            st.B=cat(2, -1*(PAR.subs(sb).cond(c).epiorg), [0 0 0 1 1 1 0 0 0]);
            %-----------------------------------------------------------------------
            mat = spm_matrix(st.B);
            if det(mat)<=0
                spm('alert!','This will flip the images',mfilename,0,1);
            end;
            Mats = zeros(4,4,size(P,1));
            spm_progress_bar('Init',size(P,1),'Reading current orientations',...
                'Images Complete');
            for i=1:size(P,1),
                Mats(:,:,i) = spm_get_space(P(i,:));
                spm_progress_bar('Set',i);
            end;
            spm_progress_bar('Init',size(P,1),'Reorienting images',...
                'Images Complete');
            for i=1:size(P,1),
                spm_get_space(P(i,:),mat*Mats(:,:,i));
                spm_progress_bar('Set',i);
            end;
            spm_progress_bar('Clear');
        end
    end

    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'seting epi done')  %-#
    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb} );
    fprintf('\r%-40s  %30s',str,' ')
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...seting T1')  %-#

    if ~isempty(PAR.subs(sb).t1org)
        st.B=cat(2, -1*PAR.subs(sb).t1org, [0 0 0 1 1 1 0 0 0]);
        mat = spm_matrix(st.B);
        P=[];
        Ptmp=spm_select('FPList', char(PAR.structdir{sb}), ['^' PAR.structprefs '\w*.img$']);
        P=strvcat(P,Ptmp);
        if strcmp(P(1,:),'/'),continue;end;
        Mats = zeros(4,4,size(P,1));
        for i=1:size(P,1),
            Mats(:,:,i) = spm_get_space(P(i,:));
        end;
        for i=1:size(P,1),
            spm_get_space(P(i,:),mat*Mats(:,:,i));
        end;
    end
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'seting T1  done')  %-#
end

Realign the images

In [ ]:
disp('We are now realigning raw EPI images for all subjects, just wait....');
% load spm defaults
global defaults;
spm_defaults;

% Get realignment defaults
defs = defaults.realign;

% Flags to pass to routine to calculate realignment parameters
% (spm_realign)

%as (possibly) seen at spm_realign_ui,
% -fwhm = 5 for fMRI
% -rtm = 0 for fMRI
% for this particular data set, we did not perform a second realignment to the mean
% the coregistration between the reference control and label volume is also omitted
reaFlags = struct(...
    'quality', defs.estimate.quality,...  % estimation quality
    'fwhm', 5,...                         % smooth before calculation
    'rtm', 1,...                          % whether to realign to mean
    'PW',''...                            %
    );

% Flags to pass to routine to create resliced images
% (spm_reslice)
resFlags = struct(...
    'interp', 1,...                       % trilinear interpolation
    'wrap', defs.write.wrap,...           % wrapping info (ignore...)
    'mask', defs.write.mask,...           % masking (see spm_reslice)
    'which',2,...                         % write reslice time series for later use
    'mean',1);                            % do write mean image


% dirnames,
% get the subdirectories in the main directory
for sb =1:PAR.nsubs % for each subject
    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb} );
    fprintf('\r%-40s  %30s',str,' ')
    P=[];
    for c=1:PAR.ncond
        % get files in this directory
        Ptmp=spm_select('ExtFPList',PAR.condirs{sb,c},['^' PAR.confilters{c} '\w*\.img$'],1:200);
        P=strvcat(P,Ptmp);
                  Ptmp=spm_select('FPList', char(PAR.M0dirs{sb,c}), ['^' PAR.M0filters{c} '\w*\.img$']);
        P=strvcat(P,Ptmp);
        
    end
    spm_realign(P, reaFlags);
    % Run reslice
    spm_reslice(P, resFlags);

end


Coregister the images

In [ ]:
% SPM2 batch file to do coregistration
% !!!!!(of structural to mean)


disp('Coregistration between structural images and the functional images for all subjects, it takes a while....');
global defaults;
spm_defaults;
flags = defaults.coreg;

% dirnames,
% get the subdirectories in the main directory
for s = 1:length(PAR.subjects) % for each subject
    %take the dir where the mean image (reslice) is stored (only first condition)
    sprintf('\nNow coregister %s''s data\n',char(PAR.subjects{s}))
    dir_fun = PAR.condirs{s,1};
    %take the structural directory
    dir_anat = PAR.structdir{s};
    % get mean in this directory
    %PG - Tar(G)et image, NEVER CHANGED
    %PF - Source image, transformed to match PG
    %PO - (O)ther images, originally realigned to PF and transformed again to PF

    % TARGET
    % get (NOT skull stripped structural from) Structurals
    %PG = spm_get('Files', dir_anat,[PAR.structprefs '*.img']);
    PG=spm_select('FPList',PAR.structdir{s},['^' PAR.structprefs '\w*\.img$']);
    PG=PG(1,:);
    VG = spm_vol(PG);

    %SOURCE
    PF = spm_select('FPList', dir_fun, ...
        ['^mean' PAR.confilters{1} '\w*\.img$']);
    %   PF = spm_get('files', dir_fun, ...
    % 		['r' PAR.confilters{1} '*img']);
    PF=PF(1,:);
    VF = spm_vol(PF);

  
    clear PO;
    PO=[];

    num=0;

    for c=1:PAR.ncond
        % get files in this directory
       % Ptmp=spm_get('files', PAR.condirs{s,c}, ['r*img']);
        Ptmp=spm_select('FPList', char(PAR.condirs{s,c}), ['^r\w.*img']);

        PO=strvcat(PO,Ptmp);
        
       Ptmp=spm_select('FPList', char(PAR.M0dirs{sb,c}),  ['^r\w.*img']);
        
        
          PO=strvcat(PO,Ptmp);
    end

    %   ana_dir = fullfile(PAR.root,PAR.subjects{s},PAR.ana_dir);
    %   confile=spm_select('FPList',ana_dir,['^con_00\w*\.img$']);
    %   PO=strvcat(PO,confile);
    %PO = PF; --> this if there are no 'other' images
    if isempty(PO) | PO=='/'
        PO=PF;
    else
        PO = strvcat(PF,PO);
    end

    %do coregistration
    %this method from spm_coreg_ui.m
    %get coregistration parameters
    x  = spm_coreg(VG, VF,flags.estimate);

    %get the transformation to be applied with parameters 'x'
    M  = inv(spm_matrix(x));
    %transform the mean image
    %spm_get_space(deblank(PG),M);
    %in MM we put the transformations for the 'other' images
    MM = zeros(4,4,size(PO,1));
    for j=1:size(PO,1),
        %get the transformation matrix for every image
        MM(:,:,j) = spm_get_space(deblank(PO(j,:)));
    end
    for j=1:size(PO,1),
        %write the transformation applied to every image
        %filename: deblank(PO(j,:))
        spm_get_space(deblank(PO(j,:)), M*MM(:,:,j));
    end
end


Smooth the images

In [ ]:
% Get subject etc parameters
disp('Smoothing the realigned functional images, it is quick....');
org_pwd=pwd;
% dirnames,
% get the subdirectories in the main directory
for sb = 1:PAR.nsubs % for each subject

    str   = sprintf('sub #%3d/%3d: %-5s',sb,PAR.nsubs,PAR.subjects{sb});
    fprintf('\r%-40s  %30s',str,' ')
    fprintf('%s%30s',repmat(sprintf('\b'),1,30),'...smoothing')  %-#

    for c=1:PAR.ncond
        %P=[];
        %dir_func = fullfile(PAR.root, PAR.subjects{s},PAR.sesses{ses},PAR.condirs{c});
        P=spm_select('FPList', PAR.condirs{sb,c}, ['^r\w*\.img$']);
        sprintf('     There are %u realigned images in this directory.',size(P,1))


        Ptmp=spm_select('FPList', char(PAR.M0dirs{sb,c}),  ['^r\w.*img']);

        P=strvcat(P,Ptmp);

        %take image per image smooth
        for im=1:size(P,1)
            %this is actual image
            Pim=P(im,:);
            %this the new image name
            
            Qim=fullfile(fileparts(Pim),['s' spm_str_manip(Pim,'dt')]);
            %now call spm_smooth with kernel defined at PAR
            spm_smooth(Pim,Qim,PAR.FWHM);
        end
        cd(PAR.condirs{sb,c});

    end

end
cd(org_pwd);

Create masks for perfusion signals

In [ ]:

for sb = 1:PAR.nsubs % for each subject
    clear P;
    for c=1:PAR.ncond
        % creating a mask image for removing background
        dir_fun = PAR.condirs{sb,1};
        PF=spm_select('ExtFPList',PAR.condirs{sb,c},['^mean' PAR.confilters{1} '\w*\.img$'],1:200);
        if size(PF,1)<1
            fprintf('No mean images for subject %s!\n',PAR.subjects{sb});
            continue;
        end
        PF=PF(1,:);
        vm=spm_vol(PF);
        dat=spm_read_vols(vm);
        mask=dat>0.2*max(dat(:));
        vo=vm;
        [path,name,ext]=fileparts(PF);
        vo.fname=fullfile(path,'mask_perf_cbf.img');
        vo=spm_write_vol(vo,mask);
    end
end

Perfusion calculation

In [ ]:
for sb = 1:PAR.nsubs % for each subject

    sprintf('Calculate perfusion and CBF signals for subject #%g ... %g subjects left...\n',sb,length(PAR.subjects)-sb)
    for c=1:PAR.ncond

        P=[];
        %ptmp=spm_get('files', PAR.condirs{s,1}, [PAR.subjects{s} '*' PAR.confilters{1} '*img']);
        %P=ptmp(1,:);
        % creating a mask image for removing background
        maskimg = spm_select('FPList', PAR.condirs{sb,c},    ['^mask_perf_cbf\w*\.img$']);
        Filename=spm_select('ExtFPList', PAR.condirs{sb,c}, ['^sr' PAR.confilters{c} '\w*\.img$']);


        Ptmp=spm_select('FPList', char(PAR.M0dirs{sb,c}), ['^sr' PAR.M0filters{c} '\w*\.img$']);
        if  length(deblank( Ptmp (1,:)))~=1
            M0img= Ptmp (1,:);

        end

        %asl_perf_subtract(Filename,FirstimageType, SubtractionType,...
        %   SubtractionOrder,Flag,
        %Timeshift,AslType,labeff,MagType,
        %   Labeltime,Delaytime,Slicetime,TE,M0img,M0seg,maskimg)
        
        % These are the values that impact the actual calculation of the
        % perfusion 
        FirstImage=1; % What is the first image? Control or label
        SubtractionType = 0; 
        SubtractionOrder=1;
        % Flag is more complex, these settings ask what outputs you would
        % like to save and create
        Flag = [1 1 1 0 0 1 0]; % For PASL
        % Flag = [1 1 1 0 0 1 0 1 1]; % For pCASL
        Timeshift = 0.5;
        AslType = 0; % Is the ASL PASL, CASL, or pCASL? 0=PASL, 1=CASL, 2=pCASL 
        labeff=0.95; % What is the label efficiency? 
        MagType = 1; % What field strength? 0 = 1.5T, 1=3T
        Labeltime= 2; % What is the labeling time?
        DelayTime = 1.5; % What is the post labeling delay? 
        SliceTime = 45; % To correct for slice timing delays, what is the slice time (only for 2D acquisitions
        TE=17; % What is the TE for the acquisition?
   
        asl_perf_subtract(Filename, FirstImage, SubtractionType, ...
            SubtractionOrder,     Flag,...
            Timeshift,     AslType,      labeff, MagType,...
            Labeltime, DelayTime, SliceTime, TE, M0img, [], maskimg);
        fprintf('\n%40s%30s','',' ');
    end
end

If you would like to see the resulting CBF images, then this is the method that would be used to load the image to view. This would require adjustment.

In [ ]:
CBF=load_nii('../sub1/ASL_S1/meanCBF_0_srASL_S1118.hdr')

In [ ]:
imagesc(CBF.img(:,:,9))